# Домашнее задание с семинара 5

## Анвардинов Шариф, Скоробогатов Денис, Царькова Анастасия

## Данные и нормализация

Возьмем случайные 100 текстов из коллекции статей NIPS (Neural Information Processing Systems) &mdash; https://www.kaggle.com/benhamner/nips-papers/data.

In [1]:
import heapq
import random
import pandas

import nltk
import nltk.stem
import nltk.corpus

In [2]:
random.seed(42)
data = random.sample(pandas.read_csv('papers.csv')['paper_text'].tolist(), 100)

Статьи на английском, поэтому для лемматизации воспользуемся wordnet из nltk.

In [3]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /home/pyos/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pyos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

`nltk.word_tokenize` разбивает сокращения наподобие "mustn't" на токены "must" и "n't", но при этом список стоп-слов для английского языка содержит куски вроде "mustn" и "t". Апострофы поэтому заменяем на пробелы.

In [4]:
%%time
lm = nltk.stem.WordNetLemmatizer()
sw = nltk.corpus.stopwords.words('english')
normed = [' '.join(lm.lemmatize(w) for w in nltk.word_tokenize(t.lower().replace("'", " ")) if w not in sw) for t in data]

CPU times: user 9.36 s, sys: 72 ms, total: 9.43 s
Wall time: 9.43 s


In [5]:
sum(len(x.split()) for x in normed)

416053

## 1. TextRank

`pip install gensim`

In [6]:
import gensim.summarization

In [9]:
%%time
textrank = {}
for text in normed:
    for i, word in enumerate(gensim.summarization.keywords(text).split('\n'), 1):
        textrank[word] = textrank.get(word, 0) + 1. / i

CPU times: user 2min 43s, sys: 11min 26s, total: 14min 10s
Wall time: 1min 10s


In [13]:
for word, weight in heapq.nlargest(20, textrank.items(), key=lambda p: p[1]):
    print(weight, word, sep='\t')

7.683715632278051	modeling
6.671925957352868	algorithm
4.734110093687647	learn
4.557393777439278	modeled
3.879231016731017	algorithmic
3.4032746161537397	learned
3.3278941071923533	model
3.204993801676846	network
3.0604089320144423	setting
2.9828754578754575	learns
2.840051438263003	trained
2.83718517860182	learning
2.7910931174089066	modelling
2.765129277408277	set
2.541970606569544	estimated
2.3980212008638335	state
2.338760396012371	compute
2.326859272518663	computing
2.263645981844416	training
2.2372757695126118	functional


Wordle требует Java-плагин для браузера. Чтобы избежать его установки, воспользуемся аналогичным сервисом https://www.wordclouds.com. Для построения облака нам потребуется tsv-файл со строками (целочисленный вес, слово или фраза). Коэффициенты подобраны вручную для оптимальных эстетических качеств визуализации.

In [79]:
with open('textrank.txt', 'w') as out:
    for word, weight in heapq.nlargest(100, textrank.items(), key=lambda p: p[1]):
        print(int(3 * weight * 1.2 ** weight), word.replace(' ', '~'), sep='\t', file=out)

![](textrank.png)

Смысл такой визуализации, честно говоря, не очевиден. Тем не менее, выделенные ключевые слова вполне подходят под тематику конференции.

## 2. RAKE

`pip install python-rake`

In [19]:
import RAKE

In [67]:
%%time
rake = {}
rake_object = RAKE.Rake(sw)
for text in normed:
    for i, (word, weight) in enumerate(rake_object.run(text), 1):
        if weight > 0:
            rake[word] = rake.get(word, 0) + weight / i

CPU times: user 4.83 s, sys: 0 ns, total: 4.83 s
Wall time: 4.83 s


In [68]:
for word, weight in heapq.nlargest(20, rake.items(), key=lambda p: p[1]):
    print(weight, word, sep='\t')

3.6651135295703305	e
3.119121367521368	ac
2.851336453959378	however
2.71758855275812	paper
2.2300233948669894	g
1.9193139733266753	edu
1.8209148349801705	mit
1.5833333333333333	arizona
1.5558873710591428	inc
1.5522520391989163	particular
1.3782773628244749	example
1.3465651650799269	b
1.341868217173073	x
1.3328144078144077	stanford
1.2577109495483587	california institute technology
1.125	ucl
1.1225401763331107	p
1.03607648919574	st
1.0114942528735633	hughes
1.0058780260830875	j


Ничего хорошего не вышло &mdash; выделились в основном одиночные символы (видимо, обозначения из формул), домены первого уровня в адресах авторов, да несколько популярных слов ("example", "particular", "paper").

## 3. Меры ассоциации биграм

Будем применять все меры сразу в линейной комбинации.

In [58]:
import nltk.collocations

In [69]:
%%time
bigrams = {}
measures = nltk.collocations.BigramAssocMeasures()
for text in normed:
    finder = nltk.collocations.BigramCollocationFinder.from_words(w for w in text.split() if w.isalnum())
    finder.apply_freq_filter(5)
    for measure, w in {measures.raw_freq: 1, measures.student_t: 1, measures.pmi: 1, measures.likelihood_ratio: 1, measures.chi_sq: 1}.items():
        for bigram, score in finder.score_ngrams(measure):
            bigrams[' '.join(bigram)] = bigrams.get(' '.join(bigram), 0) + score

CPU times: user 908 ms, sys: 0 ns, total: 908 ms
Wall time: 907 ms


In [70]:
for word, weight in heapq.nlargest(20, bigrams.items(), key=lambda p: p[1]):
    print(weight, word, sep='\t')

48160.182377200596	et al
26972.252254864212	machine learning
17419.444892701656	neural network
13537.78399558378	arxiv preprint
13305.721693496747	information processing
12639.277159118326	neural information
8585.11721438285	advance neural
8416.304119794222	fixed point
8188.8857060570535	processing system
8067.827774796364	sample complexity
7250.012599117737	monte carlo
7005.426958695783	standard deviation
6848.115285344084	upper bound
6747.228226716034	dynamic programming
6672.4962010405825	support vector
6513.642299393167	indian buffet
6437.789075463936	reinforcement learning
6349.209189542107	gradient descent
6061.768242062847	random field
5891.006442047586	logistic regression


In [80]:
with open('bigrams.txt', 'w') as out:
    for word, weight in heapq.nlargest(100, bigrams.items(), key=lambda p: p[1]):
        print(int(weight ** 0.5 - 50), word.replace(' ', '~'), sep='\t', file=out)

![](bigrams.png)

Среди биграм лучше всего выделились самые популярные (например, "et al" &mdash; "и др. [авторы]"). Явно надо подправить коэффициенты при комбинировании метрик. В список так же попали некоторые термины из машинного обучения, так что в целом это можно считать как успех.

## 4. TF-IDF

`pip install scikit-learn`

In [81]:
import sklearn.feature_extraction.text

In [103]:
%%time
tfidf = {}
tfidf_transform = sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(1,2))
tfidf_data = tfidf_transform.fit_transform(normed).todense()
tfidf_phrases = tfidf_transform.get_feature_names()
for row in tfidf_data:
    row = row.tolist()[0]
    for phrase, score in sorted(zip(range(len(row)), row), key=lambda p: p[1]):
        tfidf[tfidf_phrases[phrase]] = tfidf.get(tfidf_phrases[phrase], 0) + score

CPU times: user 19.9 s, sys: 572 ms, total: 20.5 s
Wall time: 20.5 s


In [104]:
for word, weight in heapq.nlargest(20, tfidf.items(), key=lambda p: p[1]):
    print(weight, word, sep='\t')

3.720232516137132	algorithm
3.598820523903939	model
3.014689996652276	network
2.7597894925855226	learning
2.5227077981066586	data
2.3625972901459416	set
2.3596470748711837	function
2.286311807170011	matrix
2.2595592999802037	image
2.1263697929274428	state
2.0785566158930284	time
1.956045974226798	10
1.885068370904615	distribution
1.8794562581906187	input
1.844576854104746	training
1.8119962758633	using
1.779566947039452	problem
1.7400146896722173	xi
1.7389578724681898	error
1.737494316957606	figure


In [106]:
with open('tfidf.txt', 'w') as out:
    for word, weight in heapq.nlargest(100, tfidf.items(), key=lambda p: p[1]):
        print(int(3 * weight * 1.3 ** weight), word.replace('_', '~'), sep='\t', file=out)

![](tfidf.png)

Ни одной биграммы в топ 100 не попало.